In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [40]:
from collections import Counter
import math

### NLP imports

In [84]:
from gensim.models import FastText

## Constand variables

In [4]:
PATH_TO_TRAIN = "../../data/train.tsv"

In [13]:
column_names = ['context_id', 'context_2', 'context_1', 
                           'context_0', 'reply_id', 'reply', 'label', 'confidence']

# Data

In [118]:
data = pd.read_csv(PATH_TO_TRAIN, error_bad_lines=False, sep = '[  . ? , !]?\t', 
                   header=None)

/Users/kitashov/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [119]:
data.head()

,0,1,2,3,4,5,6,7
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу","ладно , повесь трубку",0,не могу,good,0.875352
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу","ладно , повесь трубку",1,"нет , звонить буду я",neutral,0.900968
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу","ладно , повесь трубку",2,"слушай , я не мог уйти",bad,0.884320
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу","ладно , повесь трубку",3,я не прекращу звонить,good,0.982530
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу","ладно , повесь трубку",4,я звоню им,good,0.838054


In [120]:
data.columns = column_names

In [121]:
data.head(50)

,context_id,context_2,context_1,context_0,reply_id,reply,label,confidence
0,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу","ладно , повесь трубку",0,не могу,good,0.875352
1,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу","ладно , повесь трубку",1,"нет , звонить буду я",neutral,0.900968
2,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу","ладно , повесь трубку",2,"слушай , я не мог уйти",bad,0.884320
3,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу","ладно , повесь трубку",3,я не прекращу звонить,good,0.982530
4,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу","ладно , повесь трубку",4,я звоню им,good,0.838054
5,22579918886,"кликни на меня а потом на надпись "" видео - зв...","о , я тебя вижу","ладно , повесь трубку",5,просто повесь трубку,bad,0.955718
6,50117448291,бывало и получше,"слушайте , мы с женой . . . совсем заблудились","я пытаюсь добраться до юджина , но , кажется ,...",0,едете,bad,0.909115
7,50117448291,бывало и получше,"слушайте , мы с женой . . . совсем заблудились","я пытаюсь добраться до юджина , но , кажется ,...",1,едем,bad,0.984621
8,50117448291,бывало и получше,"слушайте , мы с женой . . . совсем заблудились","я пытаюсь добраться до юджина , но , кажется ,...",2,"куда же вы едете , месье",good,0.833307
9,50117448291,бывало и получше,"слушайте , мы с женой . . . совсем заблудились","я пытаюсь добраться до юджина , но , кажется ,...",3,"берите бутылку , а мы такси",good,0.800272


In [106]:
data.loc[49, 'context_2']

nan

## Exploration

### Number of utteranses in the dialog

In [108]:
number_of_conversations = len(data.context_id.unique())
print (number_of_conversations)

16526


In [109]:
Counter(data.context_2.isnull())

Counter({False: 46587, True: 46717})

## Get all unique words

In [125]:
def get_unique_column_words(dataframe, column_name):
    return list(dataframe[column_name].str.strip().split(' ', expand=True).stack().unique())

In [135]:
def get_vocab(dataframe, filename, column_names=['context_2', 'context_1', 'context_0', 'reply']):
    words = []
    for name in column_names:
        print ('getting unique words from ' + name)
        words += list(dataframe[name].str.split(' ', expand=True).stack().unique())
        words = [word.strip() for word in words]
    unique_words = set(words)
    np.save(filename, unique_words)
    return unique_words

In [136]:
train_vocab = get_vocab(data, 'train_vocab.npy')

getting unique words from context_2
getting unique words from context_1
getting unique words from context_0
getting unique words from reply


In [130]:
train_vocab

{'',
 'мягкая',
 'угнать',
 'прямее',
 'гадко',
 'горчицы',
 'летел',
 'расплакалась',
 'событиям',
 'саванна',
 'железная',
 'шерри',
 'завтрашний',
 'прервана',
 'нельсона',
 'перекинуться',
 'хаббл',
 'побережью',
 'администрация',
 'страшная',
 'каково',
 'затруднит',
 'обогнал',
 'одиночный',
 'отравила',
 'выпустим',
 'мачо',
 'заплатим',
 'пёс',
 'казнил',
 'живёт',
 'поплавать',
 'жути',
 'уведомления',
 'переключились',
 'порядок',
 'относящееся',
 'закрываетесь',
 'у',
 'идеал',
 'веские',
 'маршалом',
 'заложники',
 'дому',
 'единственную',
 'арестованы',
 'голый',
 'викки',
 'платье',
 'сексуальна',
 'мика',
 'приговорят',
 'данн',
 'готовые',
 'планет',
 'твоем',
 'left',
 'золотце',
 'радостный',
 'заслоняет',
 'оставалась',
 'сияет',
 'напоминает',
 'закажете',
 'xbox',
 'достигли',
 'подох',
 'подкован',
 'планета',
 'гмм',
 'наде',
 'палки',
 'яркая',
 'столбу',
 'заключу',
 'передам',
 'выяснила',
 'камбоджу',
 'отсидел',
 'английскому',
 'мышка',
 'мотоцикл',
 'шелдо

In [4]:
'€' in '€rs'

True

In [10]:
word = '€rs'
if '€' in word:
    print (word.index('€'))
    word = word[:word.index('€')] + 'я' + word[word.index('€') + 1:]
    print (word)

0
яrs


In [13]:
set(['a', 'b', 'c']) - set(['a'])

{'b', 'c'}

In [ ]:
"../../data/train_context_ids.npy"